# Roll decay test parameter sensitivity many


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 200)
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 15, 7

import os
import copy
from scipy.optimize import curve_fit

from rolldecay.simulation import  simulate
#import rolldecay.parameter_identification as parameter_identification
import rolldecay.read_funky_ascii
import inspect
from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.norwegian_estimator import NorwegianEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
#from rolldecay.equations_lambdify import calculate_acceleration, calculate_velocity
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [ ]:
122/(19*2*3.14)

In [ ]:
exclude_files = ['20084871055k.06.asc','20084871055k.03.asc'
                ]
acii_directory_path = os.path.join(rolldecay.data_path,'project1','Ascii files')
file_names = []
for file_name in os.listdir(acii_directory_path):
    file_names.append(file_name)
    
file_names = list(set(file_names) - set(exclude_files))


In [ ]:
exclude_files = ['20084871055k.06.asc','20084871055k.03.asc'
                ]
acii_directory_path = os.path.join(rolldecay.data_path,'project1','Ascii files')
file_names = []
for file_name in os.listdir(acii_directory_path):
    file_names.append(file_name)
    
file_names = list(set(file_names) - set(exclude_files))


In [ ]:
def fit(pipeline):
    
    pipelines = {}
    
    for file_name in file_names:
        
        ascii_file_path = os.path.join(acii_directory_path,file_name)
        df_raw = rolldecay.read_funky_ascii.read(ascii_path=ascii_file_path)[['phi']]
        
        # Exclude tests where roll is not big enough:
        if ((df_raw['phi'].max() < np.deg2rad(1)) |
            (df_raw['phi'].min() > -np.deg2rad(1))):
            continue
        
        
        pipeline = copy.deepcopy(pipeline)
        pipeline.fit(X=df_raw)
        pipelines[file_name] = pipeline
        
    return pipelines

In [ ]:
def predict(pipelines):
    
    df = pd.DataFrame()
    
    for file_name, pipeline in pipelines.items():
        
        estimator = pipeline[-1]
        s = pd.Series(estimator.parameters, name=file_name)
        s['score'] = estimator.score(X=estimator.X)
        s['mean_damping'] = estimator.calculate_average_linear_damping()
        df = df.append(s)
        
    return df

## Direct

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=0.4)
scaler = ScaleFactorTransformer(scale_factor=29.565)
cutter = CutTransformer(phi_max=np.deg2rad(15), phi_min=np.deg2rad(1))
offset_transformer = OffsetTransformer()
bounds = {
    'zeta':(0,np.inf),  # Only positive damping
    'd':(0,np.inf),  # Only positive damping
}
direct_estimator = DirectEstimator(bounds=bounds)

steps = [
('filter',lowpass_filter),
('scaler',scaler),
('cutter', cutter), 
('offset_transformer',offset_transformer),
('direct_estimator', direct_estimator)]
    
pipeline_direct = Pipeline(steps) # define the pipeline object.

In [ ]:
pipeline_directs = fit(pipeline=pipeline_direct)

In [ ]:
pipeline = pipeline_directs['20084871051k.01.asc']

In [ ]:
df_direct = predict(pipelines=pipeline_directs)

## Norwegian

In [ ]:
norwegian_estimator = NorwegianEstimator(bounds=bounds)

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('norwegian_estimator', norwegian_estimator)]
        
pipeline_norwegian = Pipeline(steps) # define the pipeline object.   

In [ ]:
pipeline_norwegians = fit(pipeline=pipeline_norwegian)

In [ ]:
df_norwegian = predict(pipelines=pipeline_norwegians)

## Linear method

In [ ]:
direct_linear_estimator = DirectLinearEstimator()

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('norwegian_estimator', direct_linear_estimator)]
        
pipeline_direct_linear = Pipeline(steps) # define the pipeline object.   

In [ ]:
pipeline_direct_linears = fit(pipeline=pipeline_direct_linear)

In [ ]:
df_direct_linear = predict(pipelines=pipeline_direct_linears)
df_direct_linear['d']=0

In [ ]:
mask = df_direct['score'] > 0.90
df_direct_good = df_direct.loc[mask].copy()
df_direct_linear_good = df_direct_linear.loc[mask].copy()
df_norwegian_good = df_norwegian.loc[mask].copy()



In [ ]:
df1 = df_direct_good.sort_values(by='score', ascending=False).iloc[-10:]
#df1 = df_direct_good.sort_values(by='score', ascending=False).iloc[0:10]
index = df1.index.copy()
df1.reset_index(inplace=True)

df1['method'] = 'direct'
df2 = df_direct_linear_good.loc[index].reset_index()
df2['method'] = 'direct linear'
df3 = df_norwegian_good.loc[index].reset_index()
df3['method'] = 'norwegian'


df_comparison = pd.concat([df1,df2,df3], sort=False)

In [ ]:
df_comparison.isnull().any()

In [ ]:
for column in ['zeta','d','omega0','mean_damping','score']:
    fig,ax = plt.subplots()
    
    df = df_comparison.pivot(index='method',columns='index', values=column).transpose()
    df.plot(kind='bar', ax=ax)
    ylim = ax.get_ylim()
    ax.set_ylim(df.min().min(),df.max().max())
    ax.set_title(column)


In [ ]:
file = df_direct_good.iloc[0].name
file

In [ ]:
#file = df_direct_good.iloc[0].name
file = '20084871056k.08.asc'


pipelines = [pipeline_directs, pipeline_direct_linears, pipeline_norwegians]

fig,ax = plt.subplots()
for p in pipelines:

    pipeline = p[file]
    estimator = pipeline[-1]
    estimator.plot_fit(ax=ax, model_test=False)
    print('%s:%s' % (estimator.__class__.__name__,estimator.score(X=estimator.X)))

estimator.X.plot(y='phi', label='Model test', ax=ax, style='k--');
ax.legend()


In [ ]:
pipeline = pipeline_norwegians[file]
estimator = pipeline[-1]
estimator.plot_damping()

In [ ]:
#file = df_direct_good.iloc[0].name
#file = '20084871056k.08.asc'


pipelines = [pipeline_directs, pipeline_direct_linears, pipeline_norwegians]

fig,ax = plt.subplots()
for p in pipelines:

    pipeline = p[file]
    estimator = pipeline[-1]
    estimator.plot_error(ax=ax)
    print('%s:%s' % (estimator.__class__.__name__,estimator.score(X=estimator.X)))


In [ ]:
estimator.X.plot(y='phi')

In [ ]:
df_direct_good.describe()

In [ ]:
df_norwegian_good.describe()

## The linear model is performing as good as the two quadratic for the present data
Need to investigate more ships to see if this changes...

In [ ]:
df_direct_linear_good.describe()

In [ ]:
pipeline = pipeline_directs['20084871056k.14.asc']
estimator = pipeline[-1]
estimator.X_amplitudes.plot(x='phi',y='omega0',style='.')